# Notebook 1 : Extraction et Ingestion des Données vers GCS

## Objectif

Ce notebook permet d'**extraire les données** depuis diverses sources publiques et de les **ingérer dans Google Cloud Storage (GCS)** dans la couche "bronze" (données brutes). Il constitue la première étape du pipeline ETL :

- **Extraction** : Téléchargement des données depuis les APIs publiques (Île-de-France Mobilités, Éducation Nationale, Calendrier Gouv, Open-Meteo, GeoAPI)
- **Ingestion** : Upload des données brutes vers GCS pour stockage et traitement ultérieur

## Prérequis

Avant d'exécuter ce notebook, assurez-vous d'avoir :

1. **Notebook 0 exécuté avec succès** : La connexion à GCS doit être validée
2. **Fichier `.env` configuré** à la racine du projet avec :
   - `PROJECT_ID` : l'identifiant de votre projet GCP
   - `GOOGLE_APPLICATION_CREDENTIALS` : le chemin vers votre fichier de credentials JSON
   - `BUCKET_NAME` : le nom de votre bucket GCS
3. **Service Account** avec les permissions :
   - `Storage Admin` ou `Storage Object Creator` pour GCS
4. **Packages Python installés** :
   ```bash
   pip install -r requirements.txt
   ```
   
Une fois ce notebook exécuté avec succès, vous pouvez passer au :
- **Notebook 2** : Chargement des données depuis GCS vers BigQuery (couche "silver").



## 0 - Configuration

- Chargement des variables d'environnement depuis `.env`
- Initialisation des chemins et identifiants du projet
- Création du dossier local `data/` pour le stockage temporaire


In [4]:
import json
import os
from datetime import datetime
from pathlib import Path
import sys
from zipfile import ZipFile

import pandas as pd
import requests
from dotenv import load_dotenv
from google.cloud import storage
from google.oauth2 import service_account

ROOT = Path.cwd().parent
sys.path.append(str(ROOT))

from src.gcs_utils import (
    download_parquet_from_idfm,
    upload_to_gcs,
    upload_folder_to_gcs,
)

load_dotenv()

ROOT = Path.cwd().parent
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

PROJECT_ID = os.getenv("PROJECT_ID")
SA_PATH = ROOT / os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
BUCKET_NAME = os.getenv("BUCKET_NAME")


In [5]:
# Authentification Google Cloud
creds = service_account.Credentials.from_service_account_file(SA_PATH)
storage_client = storage.Client(project=PROJECT_ID, credentials=creds)

## 1 - Données Historiques de Validations

**Source** : API Île-de-France Mobilités (`histo-validations-reseau-ferre`)

**Format** : Fichiers ZIP contenant des CSV/TXT (2015-2024)

**Processus** :
1. Récupération de la liste des fichiers via l'API
2. Téléchargement et extraction des archives ZIP
3. Upload vers GCS (filtrage `*_NB_FER.txt` et `*_NB_FER.csv` case-insensitive)


In [6]:
# 1.1 - Récupération des liens des fichiers depuis l'API ile de de France Mobilités
URL = "https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/histo-validations-reseau-ferre/records"

response = requests.get(URL, timeout=60)
response.raise_for_status()

records = response.json().get("results", [])
print(f"Nombre de fichiers trouvés: {len(records)}")

df_metadata = pd.DataFrame([
    {
        'annee': int(r['annee']),
        'filename': r['reseau_ferre']['filename'],
        'url': r['reseau_ferre']['url']
    }
    for r in records
])

print("\nFichiers disponibles:")
display(df_metadata)

Nombre de fichiers trouvés: 10

Fichiers disponibles:


,annee,filename,url
0,2017,data-rf-2017.zip,https://data.iledefrance-mobilites.fr/api/expl...
1,2019,data-rf-2019.zip,https://data.iledefrance-mobilites.fr/api/expl...
2,2020,data-rf-2020.zip,https://data.iledefrance-mobilites.fr/api/expl...
3,2021,data-rf-2021.zip,https://data.iledefrance-mobilites.fr/api/expl...
4,2018,data-rf-2018.zip,https://data.iledefrance-mobilites.fr/api/expl...
5,2016,data-rf-2016.zip,https://data.iledefrance-mobilites.fr/api/expl...
6,2022,data-rf-2022.zip,https://data.iledefrance-mobilites.fr/api/expl...
7,2024,data-rf-2024.zip,https://data.iledefrance-mobilites.fr/api/expl...
8,2015,data-rf-2015.zip,https://data.iledefrance-mobilites.fr/api/expl...
9,2023,data-rf-2023.zip,https://data.iledefrance-mobilites.fr/api/expl...


In [7]:
# 1.2 - Téléchargement des fichiers
BASE_DIR = DATA_DIR / "histo-validations-reseau-ferre"
BASE_DIR.mkdir(parents=True, exist_ok=True)

for row in df_metadata.itertuples():
    year_dir = BASE_DIR / str(row.annee)
    year_dir.mkdir(exist_ok=True)
    zip_path = year_dir / row.filename

    print(f"[...] - Téléchargement {row.annee} ({row.filename})")
    try:
        response = requests.get(row.url, timeout=300)
        response.raise_for_status()
        
        zip_path.write_bytes(response.content)
        print(f"    [OK] - Téléchargé")
        
        with ZipFile(zip_path) as zf:
            zf.extractall(year_dir)
        print(f"    [OK] - Extrait dans {year_dir}")
        
        zip_path.unlink()
        
    except Exception as e:
        print(f"[Erreur] {row.annee}: {e}")

print(f"\n[OK] - Terminé: {BASE_DIR}")

[...] - Téléchargement 2017 (data-rf-2017.zip)
    [OK] - Téléchargé
    [OK] - Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2017
[...] - Téléchargement 2019 (data-rf-2019.zip)
    [OK] - Téléchargé
    [OK] - Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2019
[...] - Téléchargement 2020 (data-rf-2020.zip)
    [OK] - Téléchargé
    [OK] - Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2020
[...] - Téléchargement 2021 (data-rf-2021.zip)
    [OK] - Téléchargé
    [OK] - Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2021
[...] - Téléchargement 2018 (data-rf-2018.zip)
    [OK] - Téléchargé
    [OK] - Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2018
[...] - Té

In [8]:
# 1.3 - Upload vers GCS
print(f"[Upload] - Upload des données historiques vers GCS...")
upload_folder_to_gcs(
    folder_path=BASE_DIR,
    storage_client=storage_client,
    bucket_name=BUCKET_NAME,
    gcs_folder="bronze", 
    gcs_subfolder="histo-validations-reseau-ferre", 
    extensions=[".csv", ".txt"]
)

[Upload] - Upload des données historiques vers GCS...
[...] - Upload du dossier histo-validations-reseau-ferre vers GCS...
[...] - Extensions filtrées: .csv, .txt
  ✓ 2022/data-rf-2022/2022_S2_PROFIL_FER.txt
  ✓ 2022/data-rf-2022/2022_S1_NB_FER.txt
  ✓ 2022/data-rf-2022/2022_S1_PROFIL_FER.txt
  ✓ 2022/data-rf-2022/2022_S2_NB_FER.txt
  ✓ 2024/2024_S1_PROFIL_FER.txt
  ✓ 2024/2024_S1_NB_FER.txt
  ✓ 2023/data-rf-2023/2023_S2_PROFIL_FER.txt
  ✓ 2023/data-rf-2023/2023_S1_NB_FER .txt
  ✓ 2023/data-rf-2023/2023_S2_NB_FER.txt
  ✓ 2023/data-rf-2023/2023_S1_PROFIL_FER.txt
  ✓ 2015/data-rf-2015/2015S2_NB_FER.csv
  ✓ 2015/data-rf-2015/2015S1_PROFIL_FER.csv
  ✓ 2015/data-rf-2015/2015S2_PROFIL_FER.csv
  ✓ 2015/data-rf-2015/2015S1_NB_FER.csv
  ✓ 2017/data-rf-2017/2017_S2_PROFIL_FER.txt
  ✓ 2017/data-rf-2017/2017_S2_NB_FER.txt
  ✓ 2017/data-rf-2017/2017S1_PROFIL_FER.txt
  ✓ 2017/data-rf-2017/2017S1_NB_FER.txt
  ✓ 2019/data-rf-2019/2019_S2_NB_FER.txt
  ✓ 2019/data-rf-2019/2019_S2_PROFIL_FER.txt
  ✓ 2019

38

## 2 - Emplacement des Gares

**Source** : API Île-de-France Mobilités (`emplacement-des-gares-idf`)

**Format** : Parquet

**Processus** : Téléchargement direct depuis l'API et upload vers GCS


In [9]:
# 2.1 - Récupération du fichier CSV depuis l'API ile de de France Mobilités
dl_path_gares = download_parquet_from_idfm('emplacement-des-gares-idf', data_dir=DATA_DIR)
upload_to_gcs(
    file_path=dl_path_gares,
    storage_client=storage_client,
    bucket_name=BUCKET_NAME,
    gcs_folder="bronze",
    gcs_subfolder="emplacement-des-gares-idf"
)

[...] - Téléchargement du fichier Parquet emplacement-des-gares-idf...
[OK] - Téléchargé: /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/emplacement-des-gares-idf/emplacement-des-gares-idf.parquet
[OK] - Taille: 0.20 MB
[...] - Upload de emplacement-des-gares-idf.parquet vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/emplacement-des-gares-idf/emplacement-des-gares-idf.parquet
[OK] - Taille: 0.20 MB


'gs://bronze-sncf-etl/bronze/emplacement-des-gares-idf/emplacement-des-gares-idf.parquet'

## 3 - Référentiel des Lignes

**Source** : API Île-de-France Mobilités (`referentiel-des-lignes`)

**Format** : Parquet

**Processus** : Téléchargement direct depuis l'API et upload vers GCS


In [10]:
# 3.1 - Téléchargement direct du fichier Parquet depuis l'API IDFM
dl_path_lignes = download_parquet_from_idfm('referentiel-des-lignes', data_dir=DATA_DIR)
upload_to_gcs(
    file_path=dl_path_lignes,
    storage_client=storage_client,
    bucket_name=BUCKET_NAME,
    gcs_folder="bronze",
    gcs_subfolder="referentiel-des-lignes"
)


[...] - Téléchargement du fichier Parquet referentiel-des-lignes...
[OK] - Téléchargé: /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/referentiel-des-lignes/referentiel-des-lignes.parquet
[OK] - Taille: 0.18 MB
[...] - Upload de referentiel-des-lignes.parquet vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/referentiel-des-lignes/referentiel-des-lignes.parquet
[OK] - Taille: 0.18 MB


'gs://bronze-sncf-etl/bronze/referentiel-des-lignes/referentiel-des-lignes.parquet'

## 4 - Arrêts

**Source** : API Île-de-France Mobilités (`arrets`)

**Format** : Parquet

**Processus** : Téléchargement direct depuis l'API et upload vers GCS


In [11]:
# 4.1 - Téléchargement direct du fichier Parquet depuis l'API IDFM
dl_path_arrets = download_parquet_from_idfm('arrets', data_dir=DATA_DIR)
upload_to_gcs(
    file_path=dl_path_arrets,
    storage_client=storage_client,
    bucket_name=BUCKET_NAME,
    gcs_folder="bronze",
    gcs_subfolder="arrets"
)

[...] - Téléchargement du fichier Parquet arrets...
[OK] - Téléchargé: /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/arrets/arrets.parquet
[OK] - Taille: 2.77 MB
[...] - Upload de arrets.parquet vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/arrets/arrets.parquet
[OK] - Taille: 2.77 MB


'gs://bronze-sncf-etl/bronze/arrets/arrets.parquet'

## 6 - Transporteurs

In [14]:
# 5.1 - Téléchargement direct du fichier Parquet depuis l'API IDFM
dl_path_transporteurs = download_parquet_from_idfm('liste-transporteurs', data_dir=DATA_DIR)
upload_to_gcs(
    file_path=dl_path_transporteurs,
    storage_client=storage_client,
    bucket_name=BUCKET_NAME,
    gcs_folder="bronze",
    gcs_subfolder="liste-transporteurs"
)

[...] - Téléchargement du fichier Parquet liste-transporteurs...
[OK] - Téléchargé: /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/liste-transporteurs/liste-transporteurs.parquet
[OK] - Taille: 0.01 MB
[...] - Upload de liste-transporteurs.parquet vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/liste-transporteurs/liste-transporteurs.parquet
[OK] - Taille: 0.01 MB


'gs://bronze-sncf-etl/bronze/liste-transporteurs/liste-transporteurs.parquet'

## 6 - Vacances
### 6.1 - Vacances Scolaires
- **Source** : Data Gouv - Éducation Nationale (`fr-en-calendrier-scolaire`)
- **Format** : CSV
- **Processus** : Téléchargement direct et upload vers GCS

In [12]:
# 5.1 - Vacances Scolaires
URL = "https://data.education.gouv.fr/explore/dataset/fr-en-calendrier-scolaire/download/?format=csv"
OUTPUT_FILE = "vacances_scolaires.csv"

vacances_dir = DATA_DIR / "vacances-scolaires"
vacances_dir.mkdir(exist_ok=True)
csv_path = vacances_dir / OUTPUT_FILE

print(f"[...] - Téléchargement des données de vacances scolaires...")
response = requests.get(URL, timeout=120)
response.raise_for_status()

csv_path.write_bytes(response.content)

file_size_mb = csv_path.stat().st_size / (1024 * 1024)
print(f"[OK] - Téléchargé: {csv_path}")
print(f"[OK] - Taille: {file_size_mb:.2f} MB")

# Upload vers GCS
upload_to_gcs(
    file_path=csv_path,
    storage_client=storage_client,
    bucket_name=BUCKET_NAME,
    gcs_folder="bronze",
    gcs_subfolder="vacances-scolaires"
)


[...] - Téléchargement des données de vacances scolaires...
[OK] - Téléchargé: /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/vacances-scolaires/vacances_scolaires.csv
[OK] - Taille: 0.23 MB
[...] - Upload de vacances_scolaires.csv vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/vacances-scolaires/vacances_scolaires.csv
[OK] - Taille: 0.23 MB


'gs://bronze-sncf-etl/bronze/vacances-scolaires/vacances_scolaires.csv'

### 6.2 - Jours Fériés
- **Source** : API Calendrier Gouv (https://calendrier.api.gouv.fr/jours-feries/)
- **Format** : JSON (un fichier par année)
- **Processus** : Téléchargement pour plusieurs années (2015-2026) et upload vers GCS

In [13]:
# 5.2 - Jours Fériés (API Gouv) - Plusieurs années
start_year = 2015
end_year = datetime.now().year + 1

feries_dir = DATA_DIR / "jours-feries"
feries_dir.mkdir(exist_ok=True)

print(f"[...] - Téléchargement des jours fériés de {start_year} à {end_year}...")

for year in range(start_year, end_year + 1):
    URL = f"https://calendrier.api.gouv.fr/jours-feries/metropole/{year}.json"
    json_path = feries_dir / f"jours_feries_{year}.json"
    
    try:
        response = requests.get(URL, timeout=60)
        response.raise_for_status()
        
        feries = response.json()
        json_path.write_text(json.dumps(feries, ensure_ascii=False, indent=2), encoding="utf-8")
        
        print(f"[OK] - {year}: {len(feries)} jours fériés")
        
        # Upload vers GCS
        upload_to_gcs(
            file_path=json_path,
            storage_client=storage_client,
            bucket_name=BUCKET_NAME,
            gcs_folder="bronze",
            gcs_subfolder="jours-feries"
        )
        
    except Exception as e:
        print(f"[Erreur] - {year}: {e}")

print(f"\n[OK] - Terminé: {feries_dir}")


[...] - Téléchargement des jours fériés de 2015 à 2026...
[OK] - 2015: 11 jours fériés
[...] - Upload de jours_feries_2015.json vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/jours-feries/jours_feries_2015.json
[OK] - Taille: 0.00 MB
[OK] - 2016: 11 jours fériés
[...] - Upload de jours_feries_2016.json vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/jours-feries/jours_feries_2016.json
[OK] - Taille: 0.00 MB
[OK] - 2017: 11 jours fériés
[...] - Upload de jours_feries_2017.json vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/jours-feries/jours_feries_2017.json
[OK] - Taille: 0.00 MB
[OK] - 2018: 11 jours fériés
[...] - Upload de jours_feries_2018.json vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/jours-feries/jours_feries_2018.json
[OK] - Taille: 0.00 MB
[OK] - 2019: 11 jours fériés
[...] - Upload de jours_feries_2019.json vers GCS...
[OK] - Uploadé: gs://bronze-sncf-etl/bronze/jours-feries/jours_feries_2019.json
[OK] - Taille: 0.00 MB
[OK] - 2020: 11 j